# Running Tune experiments with BOHB

In this tutorial we introduce BOHB, while running a simple Ray Tune experiment.
Tune’s Search Algorithms integrate with BOHB and, as a result,
allow you to seamlessly scale up a BOHB optimization
process - without sacrificing performance.

Bayesian Optimization HyperBand (BOHB) combines the benefits of Bayesian optimization
together with Bandit-based methods (e.g. HyperBand). BOHB does not rely on
the gradient of the objective function,
but instead, learns from samples of the search space.
It is suitable for optimizing functions that are non-differentiable,
with many local minima, or even unknown but only testable.
Therefore, this approach belongs to the domain of
"derivative-free optimization" and "black-box optimization".

In this example we minimize a simple objective to briefly demonstrate the usage of
BOHB with Ray Tune via `BOHBSearch`. It's useful to keep in mind that despite
the emphasis on machine learning experiments, Ray Tune optimizes any implicit
or explicit objective. Here we assume `ConfigSpace==0.4.18` and `hpbandster==0.7.4`
libraries are installed. To learn more, please refer to the
[BOHB website](https://github.com/automl/HpBandSter).

## Installing requirements:

In [1]:
!pip install -q "ray[tune]" ConfigSpace==1.2.1 hpbandster==0.7.4

  Using cached ConfigSpace-1.2.1-py3-none-any.whl
  Using cached hpbandster-0.7.4-py3-none-any.whl
  Using cached ray-2.42.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (18 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached msgpack-1.1.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (8.4 kB)
  Using cached protobuf-5.29.3-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached PyYAML-6.0.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached frozenlist-1.5.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached pandas-2.2.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl.metadata (5.8 kB)
  Usi

Click below to see all the imports we need for this example.

In [2]:
import tempfile
import time
from pathlib import Path

import ray
from ray import tune
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.search.bohb import TuneBOHB
import ConfigSpace as CS

2025-02-18 17:49:29,267	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-02-18 17:49:37,518	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Let's start by defining a simple evaluation function.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to tune
two hyperparameters, namely `width` and `height`, and `activation`.

In [3]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 1
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

Next, our `objective` function takes a Tune `config`, evaluates the `score` of your
experiment in a training loop, and uses `tune.report` to report the `score` back to Tune.

BOHB will interrupt our trials often, so we also need to {ref}`save and restore checkpoints <train-checkpointing>`.

In [4]:
def objective(config):
    start = 0
    if tune.get_checkpoint():
        with tune.get_checkpoint().as_directory() as checkpoint_dir:
            start = int((Path(checkpoint_dir) / "data.ckpt").read_text())

    for step in range(start, config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        with tempfile.TemporaryDirectory() as checkpoint_dir:
            (Path(checkpoint_dir) / "data.ckpt").write_text(str(step))
            tune.report(
                {"iterations": step, "mean_loss": score},
                checkpoint=tune.Checkpoint.from_directory(checkpoint_dir)
            )

In [5]:
ray.init(configure_logging=False)

Python version:,3.11.11
Ray version:,2.42.1


Next we define a search space. The critical assumption is that the optimal
hyperparameters live within this space. Yet, if the space is very large,
then those hyperparameters may be difficult to find in a short amount of time.

In [6]:
search_space = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
    "activation": tune.choice(["relu", "tanh"]),
}

Next we define the search algorithm built from `TuneBOHB`, constrained
to a maximum of `4` concurrent trials with a `ConcurrencyLimiter`.
Below `algo` will take care of the BO (Bayesian optimization) part of BOHB,
while scheduler will take care the HB (HyperBand) part.

In [7]:
algo = TuneBOHB()
algo = tune.search.ConcurrencyLimiter(algo, max_concurrent=4)
scheduler = HyperBandForBOHB(
    time_attr="training_iteration",
    max_t=100,
    reduction_factor=4,
    stop_last_trials=False,
)

The number of samples is the number of hyperparameter combinations
that will be tried out. This Tune run is set to `1000` samples.
(you can decrease this if it takes too long on your machine).

In [8]:
num_samples = 1000

In [9]:
num_samples = 10

Finally, we run the experiment to `min`imize the "mean_loss" of the `objective`
by searching within `"steps": 100` via `algo`, `num_samples` times. This previous
sentence is fully characterizes the search problem we aim to solve.
With this in mind, notice how efficient it is to execute `tuner.fit()`.

In [10]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        scheduler=scheduler,
        num_samples=num_samples,
    ),
    run_config=tune.RunConfig(
        name="bohb_exp",
        stop={"training_iteration": 100},
    ),
    param_space=search_space,
)
results = tuner.fit()

TypeError: UniformFloatHyperparameter.__init__() got an unexpected keyword argument 'q'

Here are the hyperparameters found to minimize the mean loss of the defined objective.

In [ ]:
print("Best hyperparameters found were: ", results.get_best_result().config)

## Optional: Passing the search space via the TuneBOHB algorithm

We can define the hyperparameter search space using `ConfigSpace`,
which is the format accepted by BOHB.

In [11]:
config_space = CS.ConfigurationSpace()
config_space.add_hyperparameter(
    CS.Constant("steps", 100)
)
config_space.add_hyperparameter(
    CS.UniformFloatHyperparameter("width", lower=0, upper=20)
)
config_space.add_hyperparameter(
    CS.UniformFloatHyperparameter("height", lower=-100, upper=100)
)
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter(
        "activation", choices=["relu", "tanh"]
    )
)

/var/folders/tp/x4qghynj1j15wqz89l2d8rz80000gp/T/ipykernel_35844/2408211672.py:2: DeprecationWarning: Please use `space.add(hyperparameter)`
  config_space.add_hyperparameter(
/var/folders/tp/x4qghynj1j15wqz89l2d8rz80000gp/T/ipykernel_35844/2408211672.py:5: DeprecationWarning: Please use `space.add(hyperparameter)`
  config_space.add_hyperparameter(
/var/folders/tp/x4qghynj1j15wqz89l2d8rz80000gp/T/ipykernel_35844/2408211672.py:8: DeprecationWarning: Please use `space.add(hyperparameter)`
  config_space.add_hyperparameter(
/var/folders/tp/x4qghynj1j15wqz89l2d8rz80000gp/T/ipykernel_35844/2408211672.py:11: DeprecationWarning: Please use `space.add(hyperparameter)`
  config_space.add_hyperparameter(


CategoricalHyperparameter(name='activation', default_value='relu', meta=None, size=2, choices=('relu', 'tanh'), weights=None, _contains_sequence_as_value=False)

In [12]:
# As we are passing config space directly to the searcher,
# we need to define metric and mode in it as well, in addition
# to Tuner()
algo = TuneBOHB(
    space=config_space,
    metric="mean_loss",
    mode="max",
)
algo = tune.search.ConcurrencyLimiter(algo, max_concurrent=4)
scheduler = HyperBandForBOHB(
    time_attr="training_iteration",
    max_t=100,
    reduction_factor=4,
    stop_last_trials=False,
)

In [13]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        scheduler=scheduler,
        num_samples=num_samples,
    ),
    run_config=tune.RunConfig(
        name="bohb_exp_2",
        stop={"training_iteration": 100},
    ),
)
results = tuner.fit()

(objective pid=36250) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/Users/rdecal/ray_results/bohb_exp_2/objective_a9b77d17_1_activation=relu,height=-75.1394,steps=100,width=10.5669_2025-02-18_17-51-14/checkpoint_000000)
(objective pid=36287) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/Users/rdecal/ray_results/bohb_exp_2/objective_20dc44ac_9_activation=relu,height=-43.2745,steps=100,width=16.3874_2025-02-18_17-51-20/checkpoint_000000) [repeated 8x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(objective pid=36296) Restored on 127.0.0.1 from checkpoint: Checkpoint(filesystem=local, path=/Users/rdecal/ray_results/bohb_exp_2/objective_a9b77d17_1_activation=relu,height=-75.1394,steps=100,width=10.5669_2025-02-18_17-51-14/checkpoint_000000)
(objective pid=36333

Remote file not found: /Users/rdecal/ray_results/bohb_exp_2/objective_a9b77d17_1_activation=relu,height=-75.1394,steps=100,width=10.5669_2025-02-18_17-51-14/result.json
Remote file not found: /Users/rdecal/ray_results/bohb_exp_2/objective_08327a70_2_activation=tanh,height=-53.6328,steps=100,width=0.4144_2025-02-18_17-51-15/result.json
Remote file not found: /Users/rdecal/ray_results/bohb_exp_2/objective_357e4037_3_activation=tanh,height=8.2965,steps=100,width=6.1596_2025-02-18_17-51-16/result.json


Here again are the hyperparameters found to minimize the mean loss of the
defined objective.

In [14]:
print(f"Best hyperparameters found were: {results.get_best_result().config}")

Best hyperparameters found were:  {'activation': np.str_('tanh'), 'height': -55.5680806062149, 'steps': 100, 'width': 9.6430071907735}


In [19]:
ray.shutdown()